# Warsztaty Python w Data Science

---
## Text Mining - część 3 z 3  

### Preprocessing
### Ekstrakcja tematów
---

## Czynniki wpływające na sukces ekstrakcji tematu

1. Jakość wstępnej obróbki
2. Róznorodnośc danych
3. Długo, długo nic ...
4. Dobór algorytmu
5. Parametryzacja algorytmu

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

In [18]:
import warnings
warnings.filterwarnings('ignore')

`import nltk` 

`nltk.download('stopwords')`

`nltk.download('wordnet')`

In [ ]:
import gzip
import pandas as pd

df = pd.read_json(
         gzip.open('data/newsgroups.json.gz', 'rt', encoding='utf-8')
)

df.head()

In [ ]:
df.target_names.unique()

In [ ]:
example = df['content'][1]
example


In [ ]:
raw_data = []

for row in df.iterrows():
    raw_data.append(row[1]['content'])

In [ ]:
raw_data[1]

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
len(stop_words)

In [ ]:
print(stop_words)

In [ ]:
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

wn_lemmatizer = WordNetLemmatizer()
    
def lemmatize(word):
    return wn_lemmatizer.lemmatize(word)

In [ ]:
lemmatize('automata')

In [ ]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    result = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            result.append('URL')
        elif token.orth_.startswith('@'):
            result.append('SCREEN_NAME')
        else:
            result.append(token.lower_)
    return result

In [ ]:
print(tokenize(example))

In [ ]:
def preprocessing(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if len(token) < 14]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatize(token) for token in tokens]
    return tokens

In [ ]:
print(preprocessing(example))

```python

# Zmień z Markdown na Code

text_data = [ preprocessing(text) for text in raw_data ]

```

---
## Pikluj co się da!
---

In [ ]:
import pickle

pickle.dump(text_data, open('data/text_data.pkl', 'wb'))

In [14]:
import pickle

text_data = pickle.load(open('data/text_data.pkl', 'rb'))

In [ ]:
import pickle
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

pickle.dump(corpus, open('data\corpus.pkl', 'wb'))
dictionary.save('data\dictionary.gensim')

In [7]:
import pickle

corpus = pickle.load(open('data\corpus.pkl', 'rb'))
dictionary = pickle.load(open('data\dictionary.gensim', 'rb'))

<>:3: DeprecationWarning: invalid escape sequence \c
<>:4: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \c
<>:4: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \c
<>:4: DeprecationWarning: invalid escape sequence \d
<ipython-input-7-29ca36d5a6f8>:3: DeprecationWarning: invalid escape sequence \c
  corpus = pickle.load(open('data\corpus.pkl', 'rb'))
<ipython-input-7-29ca36d5a6f8>:4: DeprecationWarning: invalid escape sequence \d
  dictionary = pickle.load(open('data\dictionary.gensim', 'rb'))


In [ ]:
import gensim
NUM_TOPICS = 5

# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel = gensim.models.ldamulticore.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('data/model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

```python

# Zmień z Markdown na Code

import gensim
NUM_TOPICS = 20

# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel = gensim.models.ldamulticore.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('data/model20.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)
    
```

In [5]:
import gensim

ldamodel = gensim.models.ldamodel.LdaModel.load('data/model20.gensim')

C:\Users\sesa443784\AppData\Local\Continuum\anaconda3\envs\py3k\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)
    

In [ ]:
new_doc = 'Data science is a new technology that uses statistics and computer science'
new_doc = preprocessing(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)

print(ldamodel.get_document_topics(new_doc_bow))

In [ ]:
rez = ldamodel.get_document_topics(new_doc_bow)

In [ ]:
rez.sort(key=lambda x: x[1], reverse=True)
rez

In [19]:
example_doc = preprocessing(example)
example_bow = dictionary.doc2bow(new_doc)
print(ldamodel.get_document_topics(example_bow))

NameError: name 'preprocessing' is not defined

In [12]:
import pyLDAvis
from pyLDAvis import gensim as ldavisgensim

v = ldavisgensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(v)

C:\Users\sesa443784\AppData\Local\Continuum\anaconda3\envs\py3k\lib\site-packages\joblib\numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\sesa443784\AppData\Local\Continuum\anaconda3\envs\py3k\lib\site-packages\joblib\numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


`!pip install pyldavis==2.1.0`

---
#Zadanie 1.

## Zbudowac wyszukiwarkę 
    1. Zbudowac model korpusu
    2. 